In [ ]:
# Import thư viện TensorFlow (thư viện chính cho học sâu - deep learning)
import tensorflow as tf

# Import mô hình EfficientNetB0 có sẵn từ thư viện Keras (dùng cho transfer learning)
from tensorflow.keras.applications import EfficientNetB0

# Import các lớp cần thiết để xây mô hình tùy chỉnh
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Import lớp Model để kết hợp các lớp và tạo mô hình tổng thể
from tensorflow.keras.models import Model

# Dùng để tạo và xử lý dữ liệu hình ảnh (hỗ trợ data augmentation, resize, batching,...)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import thư viện thao tác với file hệ thống
import os

# Import NumPy để xử lý dữ liệu số, mảng, ma trận (thường dùng cho dữ liệu ảnh và mô hình)
import numpy as np

# Import hàm đánh giá độ chính xác từ scikit-learn
from sklearn.metrics import accuracy_score

# OpenCV: thư viện xử lý ảnh truyền thống, dùng để đọc/viết ảnh, xử lý ảnh trước khi đưa vào mô hình
import cv2


In [ ]:
# Định nghĩa đường dẫn đến thư mục chứa dữ liệu huấn luyện và kiểm thử
train_dir = '/kaggle/input/dataset-version001/Dataset_verssion001/main'  # Thư mục chứa dữ liệu train + validation
test_dir = '/kaggle/input/dataset-version001/Dataset_verssion001/test'   # Thư mục chứa dữ liệu test (sau khi train)

# Khởi tạo ImageDataGenerator để xử lý dữ liệu ảnh
datagen = ImageDataGenerator(
    rescale=1./255,           # Chuẩn hóa pixel ảnh về khoảng [0,1]
    validation_split=0.2      # Tách 20% dữ liệu train thành validation (giữ 80% để train)
)

# Tạo generator cho tập huấn luyện (80% dữ liệu trong train_dir)
train_generator = datagen.flow_from_directory(
    train_dir,                # Đường dẫn thư mục chính
    target_size=(224, 224),   # Resize ảnh về kích thước phù hợp với EfficientNet
    color_mode='rgb',         # Ảnh màu RGB
    batch_size=64,            # Số lượng ảnh mỗi batch
    class_mode='sparse',      # Nhãn được trả về dưới dạng số nguyên (dành cho sparse_categorical_crossentropy)
    subset='training'         # Chỉ lấy phần dữ liệu dành cho train (80%)
)

# Tạo generator cho tập validation (20% còn lại trong train_dir)
val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=64,
    class_mode='sparse',
    subset='validation'       # Lấy phần validation (20%)
)


Found 9409 images belonging to 3 classes.
Found 2351 images belonging to 3 classes.


In [ ]:
# Xây dựng mô hình EfficientNetB0

# Load mô hình EfficientNetB0 đã được huấn luyện sẵn trên ImageNet, loại bỏ phần đầu ra (include_top=False)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Lấy output từ mô hình gốc và thêm các lớp đầu ra tùy chỉnh phù hợp với bài toán
x = base_model.output
x = GlobalAveragePooling2D()(x)            # Lấy trung bình toàn cục của đặc trưng không gian (global pooling)
x = Dense(1024, activation='relu')(x)      # Lớp ẩn với 1024 node, dùng ReLU để học thêm đặc trưng
predictions = Dense(3, activation='softmax')(x)  # Lớp đầu ra với 3 lớp, dùng softmax để phân loại đa lớp

# Tạo mô hình hoàn chỉnh từ đầu vào EfficientNet và đầu ra tuỳ chỉnh
model = Model(inputs=base_model.input, outputs=predictions)

# Biên dịch mô hình: dùng Adam optimizer, hàm mất mát phù hợp với nhãn dạng số nguyên, và đánh giá bằng accuracy
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Huấn luyện mô hình với dữ liệu training và validation
history = model.fit(
    train_generator,          # Dữ liệu huấn luyện
    validation_data=val_generator,  # Dữ liệu kiểm tra
    epochs=9                  # Số vòng lặp qua toàn bộ dữ liệu (epochs)
)

# Lưu mô hình sau khi huấn luyện vào file .h5 để sử dụng sau này (dùng để dự đoán hoặc fine-tune tiếp)
model.save("efficientnetb0.h5")

Epoch 1/9
148/148 ━━━━━━━━━━━━━━━━━━━━ 175s 602ms/step - accuracy: 0.9355 - loss: 0.1600 - val_accuracy: 0.3148 - val_loss: 1.1262
Epoch 2/9
148/148 ━━━━━━━━━━━━━━━━━━━━ 53s 341ms/step - accuracy: 0.9924 - loss: 0.0241 - val_accuracy: 0.3148 - val_loss: 3.5343
Epoch 3/9
148/148 ━━━━━━━━━━━━━━━━━━━━ 53s 346ms/step - accuracy: 0.9889 - loss: 0.0367 - val_accuracy: 0.3352 - val_loss: 1.6298
Epoch 4/9
148/148 ━━━━━━━━━━━━━━━━━━━━ 51s 334ms/step - accuracy: 0.9961 - loss: 0.0109 - val_accuracy: 0.7325 - val_loss: 0.7720
Epoch 5/9
148/148 ━━━━━━━━━━━━━━━━━━━━ 53s 342ms/step - accuracy: 0.9889 - loss: 0.0319 - val_accuracy: 0.9392 - val_loss: 0.2882
Epoch 6/9
148/148 ━━━━━━━━━━━━━━━━━━━━ 54s 350ms/step - accuracy: 0.9948 - loss: 0.0130 - val_accuracy: 0.9877 - val_loss: 0.0452
Epoch 7/9
148/148 ━━━━━━━━━━━━━━━━━━━━ 54s 349ms/step - accuracy: 0.9938 - loss: 0.0181 - val_accuracy: 0.9787 - val_loss: 0.3403
Epoch 8/9
148/148 ━━━━━━━━━━━━━━━━━━━━ 53s 343ms/step - accuracy: 0.9945 - loss: 0.0207 -

In [ ]:
# Hàm predict_folder dùng để dự đoán nhãn của ảnh trong một thư mục
def predict_folder(model, folder_path):
    predictions = []  # Danh sách lưu kết quả dự đoán
    true_labels = []  # Danh sách lưu nhãn thực tế

    # Duyệt qua tất cả các tệp trong thư mục
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Kiểm tra nếu tệp là ảnh
            img_path = os.path.join(folder_path, filename)  # Tạo đường dẫn đầy đủ đến ảnh
            img = cv2.imread(img_path)  # Đọc ảnh (mặc định đọc ảnh ở định dạng BGR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Chuyển ảnh từ BGR sang RGB

            # Resize ảnh về kích thước 224x224 (kích thước đầu vào yêu cầu của mô hình)
            img = cv2.resize(img, (224, 224))
            
            # Chuyển đổi kiểu dữ liệu ảnh thành float32 và chuẩn hóa giá trị ảnh (chia cho 255)
            img = img.astype('float32') / 255.0
            
            # Thêm một chiều vào ảnh để tạo batch size (model.predict yêu cầu đầu vào là 4 chiều)
            img = np.expand_dims(img, axis=0)

            # Dự đoán nhãn của ảnh với mô hình đã huấn luyện
            prediction = model.predict(img)
            
            # Lấy lớp dự đoán có xác suất cao nhất
            predicted_class = np.argmax(prediction, axis=1)[0]
            predictions.append(predicted_class)  # Lưu kết quả dự đoán

            # Lấy nhãn thực tế từ tên thư mục của ảnh
            true_label = os.path.basename(os.path.dirname(img_path))  # Lấy tên thư mục chứa ảnh (là nhãn thực tế)
            true_label = train_generator.class_indices[true_label]  # Chuyển nhãn thành số nguyên theo class_indices
            true_labels.append(true_label)  # Lưu nhãn thực tế

    return predictions, true_labels  # Trả về kết quả dự đoán và nhãn thực tế

# Load lại mô hình đã huấn luyện từ tệp "efficientnetb0.h5"
model = tf.keras.models.load_model("efficientnetb0.h5")

# Đường dẫn đến thư mục chứa các lớp ảnh cần kiểm tra (test)
folder_paths = [
    os.path.join(test_dir, 'incorrect_mask'),  # Thư mục chứa ảnh không có mặt nạ
    os.path.join(test_dir, 'with_mask'),      # Thư mục chứa ảnh có mặt nạ
    os.path.join(test_dir, 'without_mask')    # Thư mục chứa ảnh không có mặt nạ
]

# Biến lưu toàn bộ dự đoán và nhãn thực tế
all_predictions = []
all_true_labels = []

# Duyệt qua các thư mục trong tập kiểm tra và thực hiện dự đoán
for folder_path in folder_paths:
    predictions, true_labels = predict_folder(model, folder_path)  # Lấy kết quả dự đoán và nhãn thực tế
    all_predictions.extend(predictions)  # Thêm kết quả dự đoán vào danh sách
    all_true_labels.extend(true_labels)  # Thêm nhãn thực tế vào danh sách

# Tính toán độ chính xác trên toàn bộ tập kiểm tra
accuracy = accuracy_score(all_true_labels, all_predictions)

# In ra độ chính xác chung trên toàn bộ tập kiểm tra
print(f"Độ chính xác chung: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━